In [23]:
import pandas as pd
import re
from collections import defaultdict
import ast
from sklearn.preprocessing import StandardScaler

In [24]:
file_path = 'Aldf.csv'
dfAl = pd.read_csv(file_path)
file_path = 'Nidf.csv'
dfNi = pd.read_csv(file_path)
file_path = 'Tidf.csv'
dfTi = pd.read_csv(file_path)

In [26]:
import pandas as pd

# Define the file paths and corresponding metal types
file_paths = ['Aldf.csv', 'Nidf.csv', 'Tidf.csv']
metal_types = ['Aluminum', 'Nickel', 'Titanium']

# Initialize an empty list to store DataFrames
dfs = []

# Read each CSV file, add the metal type column, and append to the list
for file_path, metal_type in zip(file_paths, metal_types):
    try:
        df = pd.read_csv(file_path)
        df['Metal Type'] = metal_type  # Add a new column for metal type
        dfs.append(df)
    except Exception as e:
        print(f"Failed to process {file_path}: {e}")

# Concatenate the DataFrames along the rows
combined_df = pd.concat(dfs, axis=0)

# Reset the index if it's not meaningful anymore
combined_df.reset_index(drop=True, inplace=True)

# Reorder the DataFrame columns to make 'Metal Type' the first column
columns = ['Metal Type'] + [col for col in combined_df.columns if col != 'Metal Type']
combined_df = combined_df[columns]

# Replace all NaN values with zero
combined_df.fillna(0, inplace=True)

# Remove the 'Fatigue Strength' column
combined_df.drop('Fatigue Strength', axis=1, inplace=True)

# Optionally, display the DataFrame's columns and the first few rows to confirm
print(combined_df.columns)
print(combined_df.head())


Index(['Metal Type', 'Density', 'Young's Modulus', 'Yield Strength',
       'Tensile Strength', 'Specific Heat', 'Coefficient of Thermal Expansion',
       'Aluminum, Al', 'Chromium, Cr', 'Copper, Cu', 'Iron, Fe',
       'Magnesium, Mg', 'Manganese, Mn', 'Silicon, Si', 'Titanium, Ti',
       'Zinc, Zn', 'Zirconium, Zr', 'Vanadium, V', 'Boron, B', 'Nickel, Ni',
       'Gallium, Ga', 'Oxygen, O', 'Beryllium, Be', 'Bismuth, Bi', 'Lead, Pb',
       'Tin, Sn', 'Scandium, Sc', 'Lithium, Li', 'Silver, Ag', 'Carbon, C',
       'Cobalt, Co', 'Molybdenum, Mo', 'Niobium, Nb (Columbium, Cb)',
       'Phosphorus, P', 'Sulfur, S', 'Tungsten, W', 'Hydrogen, H',
       'Nitrogen, N', 'Yttrium, Y', 'Palladium, Pd', 'Tantalum, Ta'],
      dtype='object')
  Metal Type  Density  Young's Modulus  Yield Strength  Tensile Strength  \
0   Aluminum     2.80         0.291903        1.984999          1.931287   
1   Aluminum     2.72        -0.759661       -0.738788         -1.055390   
2   Aluminum     2.67    

In [28]:
combined_df

,Metal Type,Density,Young's Modulus,Yield Strength,Tensile Strength,Specific Heat,Coefficient of Thermal Expansion,"Aluminum, Al","Chromium, Cr","Copper, Cu",...,"Molybdenum, Mo","Niobium, Nb (Columbium, Cb)","Phosphorus, P","Sulfur, S","Tungsten, W","Hydrogen, H","Nitrogen, N","Yttrium, Y","Palladium, Pd","Tantalum, Ta"
0,Aluminum,2.80,0.291903,1.984999,1.931287,-0.984663,0.304406,91.4000,0.180,1.2000,...,0.0000,0.0,0.0,0.0000,0.0,0.000000,0.0000,0.0,0.0,0.00
1,Aluminum,2.72,-0.759661,-0.738788,-1.055390,-0.288575,0.448191,95.9000,0.200,0.3000,...,0.0000,0.0,0.0,0.0000,0.0,0.000000,0.0000,0.0,0.0,0.00
2,Aluminum,2.67,-0.284761,-0.100160,-0.229927,-0.225294,0.591976,97.7000,0.150,0.0400,...,0.0000,0.0,0.0,0.0000,0.0,0.000000,0.0000,0.0,0.0,0.00
3,Aluminum,2.68,-0.386526,0.753909,0.280360,-0.647166,0.591976,97.7000,0.150,0.1000,...,0.0000,0.0,0.0,0.0000,0.0,0.000000,0.0000,0.0,0.0,0.00
4,Aluminum,2.78,0.291903,-1.131198,-0.732709,-0.752633,0.448191,94.7000,0.060,0.1000,...,0.0000,0.0,0.0,0.0000,0.0,0.000000,0.0000,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,Titanium,4.48,0.535872,-0.424202,-0.300824,0.928242,0.635890,6.6785,6.680,0.1900,...,2.2125,0.0,0.0,0.0003,0.0,0.005870,0.1526,0.0,0.0,0.01
851,Titanium,4.70,-0.318018,1.379536,1.170751,0.199970,-0.340310,6.2535,0.900,0.0000,...,1.5140,0.0,0.0,0.0000,0.0,0.013645,0.0614,0.0,0.0,0.01
852,Titanium,4.65,0.870942,0.207106,0.145904,-0.528301,-0.618703,6.8025,3.795,0.1515,...,2.2715,0.0,0.0,0.0000,0.0,0.014785,0.0852,0.0,0.0,0.00
853,Titanium,4.82,-0.372061,-0.123579,-0.064321,0.782587,1.299016,5.6210,5.950,0.0140,...,2.9085,0.0,0.0,0.0003,0.0,0.004385,0.1514,0.0,0.0,0.00


In [29]:
# Save the modified DataFrame to a CSV file
combined_df.to_csv('Combodf.csv', index=False)

print("DataFrame has been saved as 'Combodfdf.csv'")


DataFrame has been saved as 'Combodfdf.csv'


In [30]:
combined_df.columns

Index(['Metal Type', 'Density', 'Young's Modulus', 'Yield Strength',
       'Tensile Strength', 'Specific Heat', 'Coefficient of Thermal Expansion',
       'Aluminum, Al', 'Chromium, Cr', 'Copper, Cu', 'Iron, Fe',
       'Magnesium, Mg', 'Manganese, Mn', 'Silicon, Si', 'Titanium, Ti',
       'Zinc, Zn', 'Zirconium, Zr', 'Vanadium, V', 'Boron, B', 'Nickel, Ni',
       'Gallium, Ga', 'Oxygen, O', 'Beryllium, Be', 'Bismuth, Bi', 'Lead, Pb',
       'Tin, Sn', 'Scandium, Sc', 'Lithium, Li', 'Silver, Ag', 'Carbon, C',
       'Cobalt, Co', 'Molybdenum, Mo', 'Niobium, Nb (Columbium, Cb)',
       'Phosphorus, P', 'Sulfur, S', 'Tungsten, W', 'Hydrogen, H',
       'Nitrogen, N', 'Yttrium, Y', 'Palladium, Pd', 'Tantalum, Ta'],
      dtype='object')

In [31]:
combined_df.shape

(855, 41)